<a href="https://colab.research.google.com/github/pi-aire/dataminig_sampling/blob/main/TP_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre

## Question 1



In [ ]:
#@title Give me a name {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


In [ ]:
print("Hello World")

Hello World
